# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [5]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [6]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [7]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [8]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from ./barcode_runs_upload.csv
Here are the first few entries in the Illumina runs:


,library,replicate,sample,sample_type,sort_bin,concentration,date,number_cells,R1
0,pool1,A,TiteSeq_01_bin1,TiteSeq,1.0,1.0,210816,1410589,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b1_S1_R1_001.fastq.gz; /shared/ngs/illumina/tstarr/210907_D00300_1314_BHMJK5BCX3/Unaligned/Project_tstarr/210816_s01-b1_S1_R1_001.fastq.gz
1,pool1,A,TiteSeq_01_bin2,TiteSeq,2.0,1.0,210816,533374,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b2_S2_R1_001.fastq.gz; /shared/ngs/illumina/tstarr/210907_D00300_1314_BHMJK5BCX3/Unaligned/Project_tstarr/210816_s01-b2_S2_R1_001.fastq.gz
2,pool1,A,TiteSeq_01_bin3,TiteSeq,3.0,1.0,210816,1697538,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b3_S3_R1_001.fastq.gz; /shared/ngs/illumina/tstarr/210907_D00300_1314_BHMJK5BCX3/Unaligned/Project_tstarr/210816_s01-b3_S3_R1_001.fastq.gz
3,pool1,A,TiteSeq_01_bin4,TiteSeq,4.0,1.0,210816,6497928,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b4_S4_R1_001.fastq.gz; /shared/ngs/illumina/tstarr/210907_D00300_1314_BHMJK5BCX3/Unaligned/Project_tstarr/210816_s01-b4_S4_R1_001.fastq.gz
4,pool1,A,TiteSeq_02_bin1,TiteSeq,1.0,2.0,210816,1587111,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s02-b1_S5_R1_001.fastq.gz; /shared/ngs/illumina/tstarr/210907_D00300_1314_BHMJK5BCX3/Unaligned/Project_tstarr/210816_s02-b1_S5_R1_001.fastq.gz


Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [9]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + x['replicate'] + '_' + x['sample'],
        title=lambda x: 'Illumina barcode sequencing for ' + x['library'] + x['replicate'] + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename
0,SAMN25944367,pool1A_TiteSeq_01_bin1,Illumina barcode sequencing for pool1ATiteSeq_01_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b1_S1_R1_001.fastq.gz,210816_s01-b1_S1_R1_001.fastq.gz
1,SAMN25944367,pool1A_TiteSeq_01_bin2,Illumina barcode sequencing for pool1ATiteSeq_01_bin2,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b2_S2_R1_001.fastq.gz,210816_s01-b2_S2_R1_001.fastq.gz
2,SAMN25944367,pool1A_TiteSeq_01_bin3,Illumina barcode sequencing for pool1ATiteSeq_01_bin3,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b3_S3_R1_001.fastq.gz,210816_s01-b3_S3_R1_001.fastq.gz
3,SAMN25944367,pool1A_TiteSeq_01_bin4,Illumina barcode sequencing for pool1ATiteSeq_01_bin4,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s01-b4_S4_R1_001.fastq.gz,210816_s01-b4_S4_R1_001.fastq.gz
4,SAMN25944367,pool1A_TiteSeq_02_bin1,Illumina barcode sequencing for pool1ATiteSeq_02_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/210903_D00300_1313_AHMJJTBCX3/Unaligned/Project_tstarr/210816_s02-b1_S5_R1_001.fastq.gz,210816_s02-b1_S5_R1_001.fastq.gz


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [10]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,biosample_accession,library_ID,n_files
0,SAMN25944367,lib1Delta_SortSeq_bin1,3
1,SAMN25944367,lib1Delta_SortSeq_bin2,3
2,SAMN25944367,lib2Delta_SortSeq_bin1,3
3,SAMN25944367,lib2Delta_SortSeq_bin2,3
4,SAMN25944367,lib2Delta_SortSeq_bin4,3
...,...,...,...
191,SAMN25944367,lib2Delta_TiteSeq_07_bin4,1
192,SAMN25944367,lib2Delta_TiteSeq_08_bin1,1
193,SAMN25944367,lib2Delta_TiteSeq_08_bin2,1
194,SAMN25944367,lib2Delta_TiteSeq_08_bin3,1


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [11]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2,filename3
0,SAMN25944367,pool1A_TiteSeq_01_bin1,Illumina barcode sequencing for pool1ATiteSeq_01_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,210816_s01-b1_S1_R1_001.fastq.gz,NaN,NaN
1,SAMN25944367,pool1A_TiteSeq_01_bin2,Illumina barcode sequencing for pool1ATiteSeq_01_bin2,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,210816_s01-b2_S2_R1_001.fastq.gz,NaN,NaN
2,SAMN25944367,pool1A_TiteSeq_01_bin3,Illumina barcode sequencing for pool1ATiteSeq_01_bin3,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,210816_s01-b3_S3_R1_001.fastq.gz,NaN,NaN
3,SAMN25944367,pool1A_TiteSeq_01_bin4,Illumina barcode sequencing for pool1ATiteSeq_01_bin4,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,210816_s01-b4_S4_R1_001.fastq.gz,NaN,NaN
4,SAMN25944367,pool1A_TiteSeq_02_bin1,Illumina barcode sequencing for pool1ATiteSeq_02_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,210816_s02-b1_S5_R1_001.fastq.gz,NaN,NaN


Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [12]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [13]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
